In [14]:
from langchain_openai import ChatOpenAI 
import os
from load_dotenv import load_dotenv

#This function will load all the variables from the .env file and will
#make them available in the os.environ dictionary (env variables)

load_dotenv()

if os.environ.get("OPENAI_API_KEY"):
    print("Bro API KEY Variable exists")
else :
    raise ValueError("OPENAI API KEY not found")

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser

llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

Bro API KEY Variable exists


In [28]:

from pydantic import BaseModel
from typing import Literal

class llm_schema(BaseModel):
    movie_summary_flag: Literal["positive","negative"]

llm_structured_output = llm_openai.with_structured_output(llm_schema)


In [16]:
result = llm_structured_output.invoke("This movie is good")
result.model_dump()['movie_summary_flag']

'positive'

## **CHAIN WITH Conditional Chains**

In [34]:
#TASK -1 [Prompt]

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a movie review evalautor"),
    ("human","please categorize the movie review as positive or negative : {input}")
])

In [18]:
#TASK - 2 [LLM]
llm_structured_output = llm_openai.with_structured_output(llm_schema)

In [19]:
# TASK -3 [Custom Runnable]
from langchain_core.runnables import RunnableLambda 

def pydantic_json(input:llm_schema)-> str:

    return input.model_dump()['movie_summary_flag']

pydantic_json_lambda = RunnableLambda(pydantic_json)



# **Conditional chain 1**

In [20]:
# TASK - 1 [Prompt]

linkedin_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Linkedln post generator"),
    ("human", "Create a post for the following text for linkedin: {text}")])

# Task - 2 [LLM]

llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

# TASK - 3 [Str Parser]

str_parser = StrOutputParser()

chain_linkedin = linkedin_prompt | llm_openai | str_parser


# **conditional chain 2**

In [21]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnableBranch

In [23]:
def insta_chain(text:dict):
    
    text = text["text"]

    
    # TASK - 1 [Prompt]
    insta_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a instagram post generator"),
    ("human", "Create a post for the following text for instagram: {text}")])

    # Task - 2 [LLM]
    llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

    # TASK - 3 [Str Parser]
    str_parser = StrOutputParser()

    chain_insta = insta_prompt | llm_openai | str_parser

    result = chain_insta.invoke(text)

    return result


insta_chain_runnable = RunnableLambda(insta_chain)




## **Final Orchestration**

In [36]:

conditional_chain = RunnableBranch(
    (lambda x: "positive" in x, chain_linkedin),
     insta_chain_runnable
)

final_orchestrator = prompt_template | llm_structured_output | pydantic_json_lambda | conditional_chain

In [37]:
final_orchestrator.invoke({"input": "i loved this KFG movie"})

"Positive isn't just a mood—it's a strategy.\n\nWhen we choose positivity at work, we don't ignore challenges; we face them with curiosity, resilience, and collaboration. A positive mindset helps teams move faster, turns setbacks into learning moments, and makes day-to-day work more sustainable.\n\nSmall, practical habits make a big difference:\n- Start meetings with a quick win or shout-out ✅\n- Give specific, constructive praise\n- Reframe problems as experiments or opportunities to learn\n- Celebrate progress, not just outcomes 🌱\n\nTry it for two weeks and notice the change in energy and outcomes. What's one small positive habit you can add to your workweek? Share below — I'd love to learn from your ideas.\n\n#Positivity #Leadership #Teamwork #GrowthMindset"